In [1]:
from theano.sandbox import cuda
import utils; reload(utils)
from utils import *

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


# Setup

In [4]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600901


In [5]:
text

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself to be won; and\nat present every kind of dogma stands with sad and discouraged mien--IF,\nindeed, it stands at all! For there are scoffers who maintain that it\nhas fallen, that all dogma lies on the ground--nay more, that it is at\nits last gasp. But to speak seriously, there are good grounds for hoping\nthat all dogmatizing in philosophy, whatever solemn, whatever conclusive\nand decided airs it has assumed, may have been only a noble puerilism\nand tyronism; and probably the time is at hand when it will be once\nand again understood WHAT has actually sufficed for the basis of such\

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 86


In [7]:
chars

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\x86',
 '\xa4',
 '\xa6',
 '\xa9',
 '\xab',
 '\xc3']

Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [8]:
chars.insert(0, '\0')

In [9]:
''.join(chars)

'\x00\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz\x86\xa4\xa6\xa9\xab\xc3'

In [10]:
char_indices = dict((c,i) for i,c in enumerate(chars))
indices_char = dict((i, c) for i,c in enumerate(chars))

In [11]:
idx = [char_indices[c] for c in text]

In [12]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [14]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

# 3 char model

In [17]:
c1_dat = [idx[i] for i in xrange(0, len(idx)-4, 3)]
c2_dat = [idx[i+1] for i in xrange(0, len(idx)-4, 3)]
c3_dat = [idx[i+2] for i in xrange(0, len(idx)-4, 3)]
c4_dat = [idx[i+3] for i in xrange(0, len(idx)-4, 3)]

In [20]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

In [21]:
y = np.stack(c4_dat[:-2])

In [22]:
x1

array([40, 30, 29, ..., 62, 72, 59])

In [23]:
x2

array([42, 25,  1, ..., 68,  2,  2])

In [25]:
x3

array([29, 27,  1, ..., 67, 68, 72])

In [26]:
y

array([30, 29,  1, ..., 72, 59, 62])

In [27]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [29]:
n_fac=42

In [42]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

In [43]:
n_hidden = 256

In [44]:
dense_in = Dense(n_hidden, activation='relu')
c1_hidden = dense_in(c1)
c2_dense = dense_in(c2)
c3_dense = dense_in(c3)

In [45]:
dense_hidden = Dense(n_hidden, activation='tanh')
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = merge([c2_dense, hidden_2])
hidden_3 = dense_hidden(c2_hidden)

In [46]:
c3_hidden = merge([c3_dense, hidden_3])

In [47]:
dense_out = Dense(vocab_size, activation='softmax')

In [48]:
c4_out = dense_out(c3_hidden)

In [49]:
model = Model([c1_in, c2_in, c3_in], c4_out)

In [50]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=Adam())

In [51]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c1 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
c2 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_4 (Embedding)          (None, 1, 42)         3612        c1[0][0]                         
____________________________________________________________________________________________________
embedding_5 (Embedding)          (None, 1, 42)         3612        c2[0][0]                         
___________________________________________________________________________________________

In [52]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4, verbose=2)

Epoch 1/4
12s - loss: 2.4002
Epoch 2/4
12s - loss: 2.2590
Epoch 3/4
12s - loss: 2.2079
Epoch 4/4
12s - loss: 2.1648


In [53]:
model.optimizer.lr=0.01

In [54]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4, verbose=2)

Epoch 1/4
12s - loss: 2.1332
Epoch 2/4
12s - loss: 2.1125
Epoch 3/4
12s - loss: 2.0978
Epoch 4/4
12s - loss: 2.0875


In [55]:
model.optimizer.lr = 0.000001

In [56]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4, verbose=2)

Epoch 1/4
12s - loss: 2.0789
Epoch 2/4
12s - loss: 2.0724
Epoch 3/4
12s - loss: 2.0679
Epoch 4/4
12s - loss: 2.0635


In [57]:
model.optimizer.lr = 0.01

In [58]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4, verbose=2)

Epoch 1/4
12s - loss: 2.0599
Epoch 2/4
12s - loss: 2.0566
Epoch 3/4
12s - loss: 2.0530
Epoch 4/4
12s - loss: 2.0503


In [59]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [60]:
get_next('phi')

'l'

In [61]:
get_next(' th')

'e'

In [62]:
get_next(' an')

'd'

# RNN

In [63]:
cs=8

In [64]:
c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)]
            for n in range(cs)]

In [65]:
c_out_dat = [idx[i+cs] for i in xrange(0, len(idx)-1-cs, cs)]

In [66]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [67]:
c_in_dat

[[40,
  1,
  33,
  2,
  72,
  67,
  73,
  2,
  68,
  57,
  72,
  2,
  65,
  68,
  62,
  71,
  78,
  58,
  54,
  61,
  65,
  67,
  57,
  9,
  58,
  65,
  74,
  67,
  78,
  74,
  73,
  2,
  75,
  65,
  73,
  57,
  73,
  8,
  58,
  62,
  57,
  65,
  57,
  62,
  2,
  27,
  78,
  72,
  54,
  61,
  73,
  67,
  73,
  73,
  64,
  57,
  54,
  61,
  57,
  71,
  58,
  62,
  62,
  72,
  3,
  58,
  72,
  2,
  67,
  54,
  72,
  8,
  65,
  2,
  2,
  74,
  2,
  61,
  72,
  2,
  72,
  73,
  2,
  65,
  58,
  68,
  57,
  68,
  54,
  68,
  67,
  62,
  8,
  58,
  67,
  75,
  65,
  67,
  58,
  62,
  72,
  66,
  2,
  65,
  65,
  65,
  2,
  66,
  71,
  73,
  2,
  54,
  2,
  2,
  1,
  62,
  72,
  25,
  56,
  72,
  2,
  2,
  59,
  66,
  54,
  65,
  65,
  54,
  56,
  61,
  73,
  75,
  71,
  58,
  54,
  2,
  2,
  62,
  2,
  62,
  1,
  72,
  74,
  72,
  2,
  62,
  68,
  74,
  54,
  72,
  73,
  72,
  73,
  2,
  62,
  21,
  72,
  65,
  2,
  54,
  62,
  61,
  68,
  54,
  67,
  68,
  71,
  68,
  71,
  62,
  58,
  68,


In [68]:
c_out_dat

[1,
 33,
 2,
 72,
 67,
 73,
 2,
 68,
 57,
 72,
 2,
 65,
 68,
 62,
 71,
 78,
 58,
 54,
 61,
 65,
 67,
 57,
 9,
 58,
 65,
 74,
 67,
 78,
 74,
 73,
 2,
 75,
 65,
 73,
 57,
 73,
 8,
 58,
 62,
 57,
 65,
 57,
 62,
 2,
 27,
 78,
 72,
 54,
 61,
 73,
 67,
 73,
 73,
 64,
 57,
 54,
 61,
 57,
 71,
 58,
 62,
 62,
 72,
 3,
 58,
 72,
 2,
 67,
 54,
 72,
 8,
 65,
 2,
 2,
 74,
 2,
 61,
 72,
 2,
 72,
 73,
 2,
 65,
 58,
 68,
 57,
 68,
 54,
 68,
 67,
 62,
 8,
 58,
 67,
 75,
 65,
 67,
 58,
 62,
 72,
 66,
 2,
 65,
 65,
 65,
 2,
 66,
 71,
 73,
 2,
 54,
 2,
 2,
 1,
 62,
 72,
 25,
 56,
 72,
 2,
 2,
 59,
 66,
 54,
 65,
 65,
 54,
 56,
 61,
 73,
 75,
 71,
 58,
 54,
 2,
 2,
 62,
 2,
 62,
 1,
 72,
 74,
 72,
 2,
 62,
 68,
 74,
 54,
 72,
 73,
 72,
 73,
 2,
 62,
 21,
 72,
 65,
 2,
 54,
 62,
 61,
 68,
 54,
 67,
 68,
 71,
 68,
 71,
 62,
 58,
 68,
 71,
 9,
 68,
 59,
 61,
 72,
 2,
 66,
 2,
 68,
 58,
 68,
 71,
 68,
 54,
 78,
 73,
 8,
 1,
 60,
 62,
 62,
 72,
 58,
 58,
 56,
 55,
 1,
 2,
 56,
 8,
 73,
 54,
 2,
 61,
 67,
 65,
 

In [69]:
xs

[array([40,  1, 33, ..., 72, 71, 61]),
 array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56])]

In [70]:
len(xs)

8

In [71]:
xs[0].shape

(75110,)

In [72]:
y = np.stack(c_out_dat[:-2])

In [73]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [74]:
y[:cs]

array([ 1, 33,  2, 72, 67, 73,  2, 68])

In [75]:
n_fac = 42

In [76]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [77]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [78]:
c_ins

[(c0_in, Reshape{2}.0),
 (c1_in, Reshape{2}.0),
 (c2_in, Reshape{2}.0),
 (c3_in, Reshape{2}.0),
 (c4_in, Reshape{2}.0),
 (c5_in, Reshape{2}.0),
 (c6_in, Reshape{2}.0),
 (c7_in, Reshape{2}.0)]

In [79]:
n_hidden = 256

In [80]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

- The first character of each sequence goes through dense_in(), to create our first hidden activations.
- Then for each successive layer, we combine the output of dense_in() on the next character with the output of dense_hidden() on the current hidden state, to create the new hidden state

In [81]:
hidden = dense_in(c_ins[0][1])
for i in range(1,cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

In [82]:
c_out = dense_out(hidden)

In [83]:
model = Model([c[0] for c in c_ins], c_out)

In [84]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c0_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c1_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c0_emb (Embedding)               (None, 1, 42)         3612        c0_in[0][0]                      
____________________________________________________________________________________________________
c1_emb (Embedding)               (None, 1, 42)         3612        c1_in[0][0]                      
___________________________________________________________________________________________

In [85]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=Adam())

In [86]:
model.fit(xs, y, batch_size=64, nb_epoch=12, verbose=2)

Epoch 1/12
10s - loss: 2.5329
Epoch 2/12
10s - loss: 2.2510
Epoch 3/12
10s - loss: 2.1511
Epoch 4/12
10s - loss: 2.0833
Epoch 5/12
10s - loss: 2.0280
Epoch 6/12
10s - loss: 1.9836
Epoch 7/12
10s - loss: 1.9434
Epoch 8/12
10s - loss: 1.9078
Epoch 9/12
10s - loss: 1.8766
Epoch 10/12
10s - loss: 1.8476
Epoch 11/12
10s - loss: 1.8233
Epoch 12/12
10s - loss: 1.8002


In [87]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [88]:
get_next('for thos')

' '

In [89]:
get_next('part of ')

't'

In [90]:
get_next('queens a')

'n'

# Keras RNN

In [91]:
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)

In [92]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=cs),
    SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
    Dense(vocab_size, activation='softmax')
])

In [93]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_7 (Embedding)          (None, 8, 42)         3612        embedding_input_1[0][0]          
____________________________________________________________________________________________________
simplernn_1 (SimpleRNN)          (None, 256)           76544       embedding_7[0][0]                
____________________________________________________________________________________________________
dense_10 (Dense)                 (None, 86)            22102       simplernn_1[0][0]                
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
____________________________________________________________________________________________________


In [94]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [95]:
model.fit(np.concatenate(xs,axis=1), y, batch_size=64, nb_epoch=8, verbose=2)

Epoch 1/8
12s - loss: 2.7927
Epoch 2/8
12s - loss: 2.2889
Epoch 3/8
12s - loss: 2.0739
Epoch 4/8
12s - loss: 1.9363
Epoch 5/8
12s - loss: 1.8328
Epoch 6/8
12s - loss: 1.7516
Epoch 7/8
12s - loss: 1.6885
Epoch 8/8
12s - loss: 1.6351


In [96]:
def get_next_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = np.array(idxs)[np.newaxis,:]
    p = model.predict(arrs)[0]
    return chars[np.argmax(p)]

In [97]:
get_next_keras('this is ')

't'

In [98]:
get_next_keras('part of ')

't'

In [99]:
get_next_keras('queens a')

'n'

# Returning sequences

In [100]:
c_out_dat = [[idx[i+n] for i in xrange(1, len(idx)-cs, cs)]
            for n in range(cs)]

In [101]:
ys = [np.stack(c[:-2]) for c in c_out_dat]

In [102]:
[xs[n][:cs] for n in range(cs)]

[array([[40],
        [ 1],
        [33],
        [ 2],
        [72],
        [67],
        [73],
        [ 2]]), array([[42],
        [ 1],
        [38],
        [44],
        [ 2],
        [ 9],
        [61],
        [73]]), array([[29],
        [43],
        [31],
        [71],
        [54],
        [ 9],
        [58],
        [61]]), array([[30],
        [45],
        [ 2],
        [74],
        [ 2],
        [76],
        [67],
        [58]]), array([[25],
        [40],
        [73],
        [73],
        [76],
        [61],
        [24],
        [71]]), array([[27],
        [40],
        [61],
        [61],
        [68],
        [54],
        [ 2],
        [58]]), array([[29],
        [39],
        [54],
        [ 2],
        [66],
        [73],
        [33],
        [ 2]]), array([[ 1],
        [43],
        [73],
        [62],
        [54],
        [ 2],
        [72],
        [67]])]

In [103]:
[ys[n][:cs] for n in range(cs)]

[array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67]),
 array([ 1, 33,  2, 72, 67, 73,  2, 68])]

In [104]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax', name='output')

In [105]:
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

In [106]:
outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden], mode='sum')
    # every layer now has an output
    outs.append(dense_out(hidden))

In [107]:
model = Model([inp1] + [c[0] for c in c_ins], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [108]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]),1))
zeros.shape

(75110, 42)

In [109]:
model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=12, verbose=2)

Epoch 1/12
22s - loss: 20.1023 - output_loss_1: 2.7128 - output_loss_2: 2.5662 - output_loss_3: 2.5179 - output_loss_4: 2.4792 - output_loss_5: 2.4706 - output_loss_6: 2.4584 - output_loss_7: 2.4621 - output_loss_8: 2.4350
Epoch 2/12
22s - loss: 17.8681 - output_loss_1: 2.5136 - output_loss_2: 2.3549 - output_loss_3: 2.2378 - output_loss_4: 2.1735 - output_loss_5: 2.1591 - output_loss_6: 2.1420 - output_loss_7: 2.1581 - output_loss_8: 2.1292
Epoch 3/12
22s - loss: 17.2475 - output_loss_1: 2.4973 - output_loss_2: 2.3317 - output_loss_3: 2.1775 - output_loss_4: 2.0834 - output_loss_5: 2.0556 - output_loss_6: 2.0330 - output_loss_7: 2.0471 - output_loss_8: 2.0220
Epoch 4/12
22s - loss: 16.8546 - output_loss_1: 2.4908 - output_loss_2: 2.3230 - output_loss_3: 2.1449 - output_loss_4: 2.0283 - output_loss_5: 1.9877 - output_loss_6: 1.9609 - output_loss_7: 1.9705 - output_loss_8: 1.9484
Epoch 5/12
22s - loss: 16.5877 - output_loss_1: 2.4872 - output_loss_2: 2.3171 - output_loss_3: 2.1231 - out

In [110]:
def get_nexts(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [111]:
get_nexts(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 't', ' ', 'i', 'n', ' ']

In [112]:
get_nexts(' part of')

[' ', 'p', 'a', 'r', 't', ' ', 'o', 'f']


['t', 'o', 'r', 't', 'i', 'o', 'f', ' ']

# Sequence model with Keras

In [113]:
n_hidden, n_fac, cs, vocab_size

(256, 42, 8, 86)

In [114]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, return_sequences=True, activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [115]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_8 (Embedding)          (None, 8, 42)         3612        embedding_input_2[0][0]          
____________________________________________________________________________________________________
simplernn_2 (SimpleRNN)          (None, 8, 256)        76544       embedding_8[0][0]                
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 8, 86)         22102       simplernn_2[0][0]                
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
____________________________________________________________________________________________________


In [116]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [117]:
xs[0].shape

(75110, 1)

In [118]:
x_rnn=np.stack(xs, axis=1)
y_rnn=np.expand_dims(np.stack(ys, axis=1), -1)

In [119]:
x_rnn.shape, y_rnn.shape

((75110, 8, 1), (75110, 8, 1, 1))

In [123]:
x_rnn[:,:,0].shape

(75110, 8)

In [124]:
model.fit(x_rnn[:,:,0], y_rnn[:,:,0], batch_size=64, nb_epoch=8, verbose=2)

Epoch 1/8
15s - loss: 2.4471
Epoch 2/8
15s - loss: 2.0052
Epoch 3/8
15s - loss: 1.8851
Epoch 4/8
15s - loss: 1.8241
Epoch 5/8
15s - loss: 1.7855
Epoch 6/8
15s - loss: 1.7596
Epoch 7/8
15s - loss: 1.7394
Epoch 8/8
15s - loss: 1.7238


In [125]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [126]:
get_nexts_keras(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 'n', ' ', 't', 'n', ' ']

# Stateful model with Keras

In [127]:
bs=64

In [128]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs,8)),
        BatchNormalization(),
        LSTM(n_hidden, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [129]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [130]:
mx = len(x_rnn)//bs*bs

In [133]:
mx

75072

In [132]:
x_rnn.shape

(75110, 8, 1)

In [135]:
y_rnn.shape

(75110, 8, 1, 1)

In [137]:
model.fit(x_rnn[:mx, :, 0], y_rnn[:mx, :, :, 0], batch_size=bs, nb_epoch=4, shuffle=False, verbose=2)

Epoch 1/4
43s - loss: 2.2214
Epoch 2/4
43s - loss: 1.9709
Epoch 3/4
43s - loss: 1.8968
Epoch 4/4
43s - loss: 1.8520


In [138]:
model.optimizer.lr=1e-4

In [140]:
model.fit(x_rnn[:mx, :, 0], y_rnn[:mx, :, :, 0], batch_size=bs, nb_epoch=4, shuffle=False, verbose=2)

Epoch 1/4
43s - loss: 1.8193
Epoch 2/4
43s - loss: 1.7932
Epoch 3/4
43s - loss: 1.7715
Epoch 4/4
43s - loss: 1.7528


In [141]:
model.fit(x_rnn[:mx, :, 0], y_rnn[:mx, :, :, 0], batch_size=bs, nb_epoch=4, shuffle=False, verbose=2)

Epoch 1/4
43s - loss: 1.7359
Epoch 2/4
43s - loss: 1.7205
Epoch 3/4
43s - loss: 1.7066
Epoch 4/4
43s - loss: 1.6938
